# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (93% passing vs 68%).

* As a whole, charter schools out-performed the public district schools across all metrics. This metric is highly visible in the % passing math, witch contributes favourably toward the % Overall Passing Rate. 

* The Top 5 best performing schools are all charter based schools and the worst 5 are all district. The Charter schools achived this despite having lower per student bugets compared to their district counter parts

* Math and reading scores from lower to higher grade in all schools are fairly consisent, with minor deviations.

* The number of students enrolled in district schools is more than double than that of charter schools, and the total budget is more than double as well

* Despite the higher number of students enrolled in district schools, the per student budget in district schools is only 50 dollars more per child as compared to charter schools.
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

#calculate and store the per student budget 
school_data["Per Student Budget"] = school_data["budget"] / school_data["size"]
school_data_complete["Per Student Budget"] = school_data_complete["budget"] / school_data_complete["size"]

In [2]:
#function that calculates the percentage of students who achieved the passing grade being a score of 70 or higher
#
#param: datain a series with the reading_score or math_score
# 
def calc_passing_percent(datain):
    x, y = datain[datain >= 70].count(), datain.count()
    return (x / y) * 100

In [3]:
#function that calcuates the average of the reading and math scores
#
#param: the reading and math scores 
#
def calc_average_percent(reading_score, math_score):
    return (reading_score + math_score) / 2

In [4]:
#function that returns a DataFrame by filtering the data based on the score_type param 
#
#param1: datain:  the student_data DataFrame
#param2: score_type: a string "reading_score" or "math_score"
def df_passing(datain, score_type):
    return datain[datain[score_type] >= 70].groupby("school_name").agg({score_type: "count"})

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [5]:
#use the agg on school_data to get the total count of schools and the sum of the budget
#use the agg on student_data to get the count of Student, and the mean of the math and reading scores
#concatenate the results into a single series
#then cast that series as a DataFrame
#then transpose rows to columns
summary_df = pd.DataFrame(pd.concat([school_data.agg({"school_name": "count"
                                                     , "budget": "sum"})
                                    , student_data.agg({"Student ID": "count"
                                                      , "math_score": "mean"
                                                      , "reading_score" : "mean"})
            ])).transpose()

#calc the passing reading grade and add that to the summary DataFrame
summary_df["% Passing Reading"] = calc_passing_percent(student_data.reading_score)

#calc the passing math grade and add that to the summary DataFrame
summary_df["% Passing Math"] = calc_passing_percent(student_data.math_score)

#calc the overall passing being the average of the passing math and reading scores and add that to the DataFrame
summary_df["% Overall Passing Rate"] = calc_average_percent(summary_df["% Passing Reading"], summary_df["% Passing Math"])

#renaming the columns of the DataFrame
summary_df = summary_df.rename(columns={"school_name": "Total Schools"
                           , "budget": "Total Budget"
                           , "Student ID" : "Total Students"
                           , "math_score" : "Average Math Score"
                           , "reading_score" : "Average Reading Score"
                          })

#reorder the columns
summary_df = summary_df[["Total Schools"
                    , "Total Students"
                    , "Total Budget"
                    , "Average Math Score"
                    , "Average Reading Score"
                    , "% Passing Math"
                    , "% Passing Reading"
                    , "% Overall Passing Rate"]]

#format output
summary_df["Total Schools"] = summary_df["Total Schools"].astype(int)
summary_df["Total Students"] = summary_df["Total Students"].astype(int)
summary_df["Total Students"] = summary_df["Total Students"].map("{:,}".format)
summary_df["Total Budget"] = summary_df["Total Budget"].map("${:,.2f}".format)

#display
summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [6]:
df_school_performance = pd.concat([
    #1 first DataFrame
    #1.1 get the columns we need from the school_data DataFrame
    school_data[["school_name"
                , "type"
                , "size"
                , "budget"
                , "Per Student Budget"]]
    #1.2 rename the columns
    .rename(columns={"type": "School Type"
                    , "size": "Total Students"
                    , "budget": "Total School Budget"
                   })
    #1.3. set the index
    .set_index("school_name")
    
    #2 Second DataFrame
    #2.1 get the columns we need fr om the student DataFrame groupby by school_name
    , student_data.groupby("school_name").agg({"reading_score": "mean"
                                         , "math_score": "mean"
                                         , "Student ID": "count"
                                        })
    #2.2 rename the columns
    .rename(columns={"reading_score": "Average Reading Score"
                       , "math_score": "Average Math Score"
                       , "Student ID": "Total Count of Students"
                      })
    
    #3 Third DataFrame, use the df_passing function to get a DataFrame with the passing reading scores
    , df_passing(student_data, "reading_score").rename(columns={"reading_score": "Count of Passing Reading Score"})
    
    #4 Fourth DataFrame, use the df_passing function to get a DataFrame with the passing math scores
    , df_passing(student_data, "math_score").rename(columns={"math_score": "Count of Passing Math Score"})
], axis = 1, sort=True)

#Adding column % Passing Reading
df_school_performance["% Passing Reading"] = df_school_performance["Count of Passing Reading Score"] / df_school_performance["Total Count of Students"] * 100

#Adding column % Passing Math
df_school_performance["% Passing Math"] = df_school_performance["Count of Passing Math Score"] / df_school_performance["Total Count of Students"] * 100

#Adding column % Overall Passing Rate
df_school_performance["% Overall Passing Rate"] = (df_school_performance["% Passing Reading"] + df_school_performance["% Passing Math"]) / 2

#just the columns we want to display
df_school_performance = df_school_performance[["School Type"
                                               , "Total Students"
                                               , "Total School Budget"
                                               , "Per Student Budget"
                                               , "Average Math Score"
                                               , "Average Reading Score"
                                               , "% Passing Math"
                                               , "% Passing Reading"
                                               , "% Overall Passing Rate"
                                              ]]
#apply formatting
df_school_performance["Total School Budget"] = df_school_performance["Total School Budget"].map("${:,.2f}".format)
df_school_performance["Per Student Budget"] = df_school_performance["Per Student Budget"].map("${:,.2f}".format)

#output top 5
df_school_performance.sort_values("% Overall Passing Rate", ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [7]:
#ouput bottom 5
df_school_performance.sort_values("% Overall Passing Rate", ascending=True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [8]:
#filter the student data first by grade, then groupby to get the mean of the math score, then cast from panda Series to DataFrame
grade09_ms = pd.DataFrame(student_data[student_data["grade"] == "9th"].groupby("school_name").mean()["math_score"]).rename(columns={"math_score": "9th"})
grade10_ms = pd.DataFrame(student_data[student_data["grade"] == "10th"].groupby("school_name").mean()["math_score"]).rename(columns={"math_score": "10th"})
grade11_ms = pd.DataFrame(student_data[student_data["grade"] == "11th"].groupby("school_name").mean()["math_score"]).rename(columns={"math_score": "11th"})
grade12_ms = pd.DataFrame(student_data[student_data["grade"] == "12th"].groupby("school_name").mean()["math_score"]).rename(columns={"math_score": "12th"})

#then combine the frames 
pd.concat([grade09_ms, grade10_ms, grade11_ms, grade12_ms], axis=1)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [9]:
#filter the student data first by grade, then groupby to get the mean of the reading score, then cast from panda Series to DataFrame
grade09_rs = pd.DataFrame(student_data[student_data["grade"] == "9th"].groupby("school_name").mean()["reading_score"]).rename(columns={"reading_score": "9th"})
grade10_rs = pd.DataFrame(student_data[student_data["grade"] == "10th"].groupby("school_name").mean()["reading_score"]).rename(columns={"reading_score": "10th"})
grade11_rs = pd.DataFrame(student_data[student_data["grade"] == "11th"].groupby("school_name").mean()["reading_score"]).rename(columns={"reading_score": "11th"})
grade12_rs = pd.DataFrame(student_data[student_data["grade"] == "12th"].groupby("school_name").mean()["reading_score"]).rename(columns={"reading_score": "12th"})

#then combine the frames 
pd.concat([grade09_rs, grade10_rs, grade11_rs, grade12_rs], axis=1)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
school_data_complete["Spending Ranges (Per Student)"] = pd.cut(school_data_complete["Per Student Budget"], bins=spending_bins, labels=group_names)
df_sum_scores = school_data_complete.groupby("Spending Ranges (Per Student)").sum()[["math_score", "reading_score"]].rename(columns={"math_score": "Sum of Math Score", "reading_score": "Sum of Reading Score"})
df_count_scores = school_data_complete.groupby("Spending Ranges (Per Student)").count()[["math_score", "reading_score"]].rename(columns={"math_score": "Count of Math Score", "reading_score": "Count of Reading Score"})

df_count_passing_math = pd.DataFrame(school_data_complete[school_data_complete["math_score"] >= 70].groupby("Spending Ranges (Per Student)").count()["math_score"]).rename(columns={"math_score":"Count of Passing Math"})
df_count_passing_reading = pd.DataFrame(school_data_complete[school_data_complete["reading_score"] >= 70].groupby("Spending Ranges (Per Student)").count()["reading_score"]).rename(columns={"reading_score":"Count Of Passing Reading"})
df_scores_by_school_pending = pd.concat([df_sum_scores, df_count_scores, df_count_passing_math, df_count_passing_reading], axis=1)

df_scores_by_school_pending["Average Math Score"] = df_scores_by_school_pending["Sum of Math Score"] / df_scores_by_school_pending["Count of Math Score"]
df_scores_by_school_pending["Average Reading Score"] = df_scores_by_school_pending["Sum of Reading Score"] / df_scores_by_school_pending["Count of Reading Score"]
df_scores_by_school_pending["% Passing Math"] = df_scores_by_school_pending["Count of Passing Math"] / df_scores_by_school_pending["Count of Math Score"] * 100
df_scores_by_school_pending["% Passing Reading"] = df_scores_by_school_pending["Count Of Passing Reading"] / df_scores_by_school_pending["Count of Reading Score"] * 100
df_scores_by_school_pending["% Overall Passing Rate"] = (df_scores_by_school_pending["% Passing Math"] + df_scores_by_school_pending["% Passing Reading"]) / 2
df_scores_by_school_pending[["Average Math Score"
                             , "Average Reading Score"
                             , "% Passing Math"
                             , "% Passing Reading"
                             ,"% Overall Passing Rate"
                            ]]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


## Scores by School Size

* Perform the same operations as above, based on school size.

In [11]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_data_complete["School Size"] = pd.cut(school_data_complete["size"], bins=size_bins, labels=group_names)
df_avg_scores_by_school_size = school_data_complete.groupby("School Size").mean()[["math_score", "reading_score"]].rename(columns={"math_score":"Average Math Score", "reading_score": "Average Reading Score"})
df_count_scores_by_school_size = school_data_complete.groupby("School Size").count()[["math_score", "reading_score"]].rename(columns={"math_score":"Count of Math Score", "reading_score": "Count of Reading Score"})
df_count_passing_math_scores_by_school_size =  pd.DataFrame(school_data_complete[school_data_complete["math_score"] >=70].groupby("School Size").count()["math_score"]).rename(columns={"math_score":"Count of Passing Math Score"})
df_count_passing_reading_scores_by_school_size = pd.DataFrame(school_data_complete[school_data_complete["reading_score"] >=70].groupby("School Size").count()["reading_score"]).rename(columns={"reading_score":"Count of Passing Reading Score"})
df_scores_by_school_size = pd.concat([df_avg_scores_by_school_size
                                       , df_count_scores_by_school_size
                                       , df_count_passing_math_scores_by_school_size
                                       , df_count_passing_reading_scores_by_school_size
                                      ], axis=1)
df_scores_by_school_size["% Passing Math"] = df_scores_by_school_size["Count of Passing Math Score"] / df_scores_by_school_size["Count of Math Score"] * 100
df_scores_by_school_size["% Passing Reading"] = df_scores_by_school_size["Count of Passing Reading Score"] / df_scores_by_school_size["Count of Reading Score"] * 100
df_scores_by_school_size["% Overall Passing Rate"] = (df_scores_by_school_size["% Passing Math"] + df_scores_by_school_size["% Passing Reading"]) / 2
df_scores_by_school_size[["Average Math Score"
                          , "Average Reading Score"
                          , "% Passing Math"
                          , "% Passing Reading"
                          , "% Overall Passing Rate"
                         ]]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


## Scores by School Type

* Perform the same operations as above, based on school type.

In [12]:
school_data_complete = school_data_complete.rename(columns={"type":"School Type"})
df_avg_scores_by_school_type = school_data_complete.groupby("School Type").mean()[["math_score", "reading_score"]].rename(columns={"math_score": "Average Math Score", "reading_score": "Average Reading Score"})
df_count_of_scores_by_school_type = school_data_complete.groupby("School Type").count()[["math_score", "reading_score"]].rename(columns={"math_score": "Count of Math Score", "reading_score": "Count of Reading Score"})
df_count_of_passing_math_score_by_school_type = pd.DataFrame(school_data_complete[school_data_complete["math_score"] >=70].groupby("School Type").count())[["math_score"]].rename(columns={"math_score": "Count of Passing Math Score"})
df_count_of_passing_reading_score_by_school_type = pd.DataFrame(school_data_complete[school_data_complete["reading_score"] >=70].groupby("School Type").count())[["reading_score"]].rename(columns={"reading_score": "Count of Passing Reading Score"})

scores_by_school_type = pd.concat([df_avg_scores_by_school_type
           , df_count_of_scores_by_school_type
           , df_count_of_passing_math_score_by_school_type
           , df_count_of_passing_reading_score_by_school_type
          ], axis=1)
scores_by_school_type["% Passing Math"] = scores_by_school_type["Count of Passing Math Score"] / scores_by_school_type["Count of Math Score"] * 100
scores_by_school_type["% Passing Reading"] = scores_by_school_type["Count of Passing Reading Score"] / scores_by_school_type["Count of Reading Score"] * 100
scores_by_school_type["% Overall Passing Rate"] = (scores_by_school_type["% Passing Math"] + scores_by_school_type["% Passing Reading"]) / 2

scores_by_school_type[["Average Math Score"
                       , "Average Reading Score"
                       , "% Passing Math"
                       , "% Passing Reading"
                       , "% Overall Passing Rate"
                      ]]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818


### I used these to infer a couple of things

In [13]:
school_data.groupby("type").sum()[["size", "budget"]]

,size,budget
type,,
Charter,12194,7301505
District,26976,17347923


In [14]:
school_data.groupby("type").mean()[["Per Student Budget"]]

,Per Student Budget
type,
Charter,599.500000
District,643.571429
